In [5]:
import os
import cv2
import pandas as pd
from libreface.utils import get_frames_from_video_opencv
from libreface.AU_Recognition.inference import get_au_intensities_and_detect_aus

In [6]:
video_path = "D:/ATHARV/W/CDAC/CODE/LIBREFACE_IMPLEMENT/Data/sample_video.avi"
temp_dir = "./tmp"
device = "cpu"
output_dir = "D:/ATHARV/W/CDAC/CODE/LIBREFACE_IMPLEMENT/Notebooks/results"
weights_dir = "./weights_libreface"
os.makedirs(output_dir, exist_ok=True)

In [7]:
print("Extracting frames...")
frames_df = get_frames_from_video_opencv(video_path, temp_dir=temp_dir)
frames_df = frames_df.iloc[::5]

if frames_df.empty:
    print("No frames extracted.")
    exit()

def draw_aus_on_frame(frame, aus: dict):
    y0, dy = 30, 30
    for i, (au, intensity) in enumerate(aus.items()):
        y = y0 + i * dy
        label = f"{au}: {intensity}"
        cv2.putText(frame, label, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 
                    1.2 , (0, 255, 0), 2, cv2.LINE_AA)
    return frame

# === AU detection ===
for _, row in frames_df.iterrows():
    frame_path = row['path_to_frame']
    frame = cv2.imread(frame_path)

    if frame is None:
        print(f"Could not read: {frame_path}")
        continue

    try:
        detected_aus, au_intensities = get_au_intensities_and_detect_aus(
            image_path=frame_path,
            device=device,
            weights_download_dir=weights_dir
        )
        annotated_data = {au: f"{au_intensities[au]:.2f}" for au in au_intensities}

        annotated_frame = draw_aus_on_frame(frame.copy(), annotated_data)

        frame_filename = os.path.basename(frame_path)
        out_path = os.path.join(output_dir, frame_filename)
        cv2.imwrite(out_path, annotated_frame)

        print(f"Processed: {frame_filename}")

    except Exception as e:
        print(f"Failed: {frame_path} – {e}")

print("AU detection complete.")

Extracting frames...
Processed: 0000000000.png
Processed: 0000000005.png
Processed: 0000000010.png
Processed: 0000000015.png
Processed: 0000000020.png
Processed: 0000000025.png
Processed: 0000000030.png
Processed: 0000000035.png
Processed: 0000000040.png
Processed: 0000000045.png
Processed: 0000000050.png
Processed: 0000000055.png
Processed: 0000000060.png
Processed: 0000000065.png
Processed: 0000000070.png
Processed: 0000000075.png
Processed: 0000000080.png
Processed: 0000000085.png
Processed: 0000000090.png
Processed: 0000000095.png
Processed: 0000000100.png
Processed: 0000000105.png
Processed: 0000000110.png
Processed: 0000000115.png
Processed: 0000000120.png
Processed: 0000000125.png
Processed: 0000000130.png
Processed: 0000000135.png
Processed: 0000000140.png
Processed: 0000000145.png
Processed: 0000000150.png
Processed: 0000000155.png
Processed: 0000000160.png
Processed: 0000000165.png
Processed: 0000000170.png
Processed: 0000000175.png
Processed: 0000000180.png
Processed: 000000